# Twirling with Program Sets on mitiq 

In this notebook, we look at Pauli twirling, and specifically how this can be realized using Amazon Braket. 

##

Pauli twirling is a technique used widely across quantum learning and mitigation protocols, known for it's simplicity and effectiveness. Essentially, twirling with classes of channels allows us to control and shape the noise channel. As a result, we can increase confidence in our knowledge of the channel and subsequent mitigation. 

Pauli twirling refers to twirling with elements of the Pauli group (i.e. tensor products of single-qubit Pauli matrices). When acting on a channel, the noise channel becomes diagonalized in the superoperator basis of Pauli matrices.

However, twirled channels themselves are generally not implementable. Instead, we approximate the twirling process using finite sets of twirls. 

Here, we show two ways this can be realized on Braket. The first is to use mitiq's built in Pauli Twirling module. The result is to create many different Paul variants defined for your circuits. 

We can additionally realize these using parameteric compilation. 

In [4]:
from mitiq import pt
from braket.circuits import Circuit 
from braket.circuits.observables import I,X,Y,Z
import numpy as np
import sys
import os 

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir))) # parent directory 
from braket.circuits.noises import AmplitudeDamping
from braket.circuits.noise_model import NoiseModel, GateCriteria
from braket.circuits.gates import CNot, CZ, Rz
from braket.devices import LocalSimulator
from mitiq.pt import generate_pauli_twirl_variants

from tools.observable_tools import matrix_to_pauli
from mitiq.pt.pt import generate_pauli_twirl_variants

from braket.aws import AwsDevice
from braket.program_sets import ProgramSet, CircuitBinding
from qiskit_braket_provider import to_braket, BraketProvider


In [5]:
cnot = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,0,1],
    [0,0,1,0]
])

ps = [I(),X(),Y(),Z()]

for i in ps:
    for j in ps:
        print(
            'inp -> out: ',
            matrix_to_pauli((i @ j).to_matrix())[0][1],'->',
            matrix_to_pauli(cnot @ (i @ j).to_matrix() @ cnot)[0][1])


inp -> out:  II -> II
inp -> out:  IX -> IX
inp -> out:  IY -> ZY
inp -> out:  IZ -> ZZ
inp -> out:  XI -> XX
inp -> out:  XX -> XI
inp -> out:  XY -> YZ
inp -> out:  XZ -> YY
inp -> out:  YI -> YX
inp -> out:  YX -> YI
inp -> out:  YY -> XZ
inp -> out:  YZ -> XY
inp -> out:  ZI -> ZI
inp -> out:  ZX -> ZX
inp -> out:  ZY -> IY
inp -> out:  ZZ -> IZ


### Background on Pauli Twirling Methodologies

For a gate, we are interested in twirling only the set of operations which 

For instance, let $\tilde{\mathcal{E}}_U$ denote the noise application of a unitary channel $U$. Then, we can write this in terms of a before-gate noise $\mathcal{E}_N$ composed with the unitary:

$$\tilde{\mathcal{E}}_{U} =  \mathcal{E}_U \circ \mathcal{E}_N$$

To twirl this, we ideally would apply a Pauli channel as follow:

$ \mathcal{E}_{twirl} = \sum_{\sigma \in \mathbb{P}} \mathcal{E}_U \circ \mathcal{E}_\sigma \circ \mathcal{E}_N \circ \mathcal{E}_\sigma $

However, we can't actually access the noise channel directly, and so we can instead commute (or anti-commute) these Paulis through the CNOT gate. This gives the following input and output Pauli twirls:

| $\sigma_i$ | II  | IX  | IY  | IZ  | XI  | XX  | XY  | XZ  | YI  | YX  | YY  | YZ  | ZI  | ZX  | ZY  | ZZ  |
| ---------- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| $\sigma_o$ | II  | IX  | ZY  | ZZ  | XX  | XI  | YZ  | YY  | YX  | YI  | XZ  | XY  | ZI  | ZX  | IY  | IZ |

Practically, we realize this by sampling the above input and output channel per each CNOT, and placing them before and after each Pauli. The CNOTs operations are preserved up to a global phase.

In [6]:

test = Circuit().cnot(0,1)
variants = generate_pauli_twirl_variants(test,3)
for i in variants:
    print(i)

T  : │  0  │  1  │  2  │
      ┌───┐       ┌───┐ 
q0 : ─┤ X ├───●───┤ X ├─
      └───┘   │   └───┘ 
      ┌───┐ ┌─┴─┐ ┌───┐ 
q1 : ─┤ I ├─┤ X ├─┤ X ├─
      └───┘ └───┘ └───┘ 
T  : │  0  │  1  │  2  │
T  : │  0  │  1  │  2  │
      ┌───┐       ┌───┐ 
q0 : ─┤ I ├───●───┤ Z ├─
      └───┘   │   └───┘ 
      ┌───┐ ┌─┴─┐ ┌───┐ 
q1 : ─┤ Z ├─┤ X ├─┤ Z ├─
      └───┘ └───┘ └───┘ 
T  : │  0  │  1  │  2  │
T  : │  0  │  1  │  2  │
      ┌───┐       ┌───┐ 
q0 : ─┤ X ├───●───┤ Y ├─
      └───┘   │   └───┘ 
      ┌───┐ ┌─┴─┐ ┌───┐ 
q1 : ─┤ Z ├─┤ X ├─┤ Y ├─
      └───┘ └───┘ └───┘ 
T  : │  0  │  1  │  2  │


For smaller systems, we can sometime enumerate all potential twirls, or use smaller twirling sets, but as the nubmer of unique circuits scales as $16^{N_{CNOTS}}$, this is quickly intractable. Thus, we just sample different Pauli twirled variants and reconstruct them. Again, these Pauli twirled variants all represent the same logical circuit, and act solely to homegenize the noise channel to a Pauli channel. 

In [ ]:

length_bell = 20
depth = 2
 
test = Circuit().h(0).cnot(0,1)

def ghz_layer(i,j):
    return Circuit().cnot(i,j).h(i).h(j).cnot(j,i)

for i in range(length_bell//2):
    test+= ghz_layer(0,1)

print(test)


T  : │  0  │  1  │  2  │  3  │  4  │  5  │  6  │  7  │  8  │  9  │ 10  │ 11  │ 12  │ 13  │ 14  │ 15  │ 16  │ 17  │ 18  │ 19  │ 20  │ 21  │ 22  │ 23  │ 24  │ 25  │ 26  │ 27  │ 28  │ 29  │ 30  │ 31  │
      ┌───┐             ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌───┐ ┌───┐       ┌───┐ ┌───┐ 
q0 : ─┤ H ├───●─────●───┤ H ├─┤ X ├───●───┤ H ├─┤ X ├───●───┤ H ├─┤ X ├───●───┤ H ├─┤ X ├───●───┤ H ├─┤ X ├───●───┤ H ├─┤ X ├───●───┤ H ├─┤ X ├───●───┤ H ├─┤ X ├───●───┤ H ├─┤ X ├───●───┤ H ├─┤ X ├─
      └───┘   │     │   └───┘ └─┬─┘   │   └───┘ └─┬─┘   │   └───┘ └─┬─┘   │   └───┘ └─┬─┘   │   └───┘ └─┬─┘   │   └───┘ └─┬─┘   │   └───┘ └─┬─┘   │   └───┘ └─┬─┘   │   └───┘ └─┬─┘   │   └───┘ └─┬─┘ 
            ┌─┴─┐ ┌─┴─┐ ┌───┐   │   ┌─┴─┐ ┌───┐   │   ┌─┴─┐ ┌───┐   │   ┌─┴─┐ ┌───┐   │   ┌─┴─┐ ┌───┐   │   ┌─┴─┐ ┌───┐   │   ┌─┴─┐ ┌───┐   │   ┌─┴─┐ ┌───┐   │   ┌─┴─┐ ┌───┐   │   ┌─┴─┐ ┌───┐   │   
q1 : 

In [8]:
pt_mitiq = generate_pauli_twirl_variants(test, num_circuits=1)
pt_mitiq_verb = to_braket(pt_mitiq, optimization_level=1, basis_gates=["r","cz"],qubit_labels=[1,2])



/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:853: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn("The Qiskit circuit contains barrier instructions that are ignored.")
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:903: UserWarning: Device does not support global phase; global phase of 4.71238898038469 will not be included in Braket circuit
  warnings.warn(


## More on Measurement Twirling

TODO: fill out details from before. 

### References